In [5]:
import json
import pandas as pd
import re
import nltk
import torch
import gc
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

from transformers import (
    RobertaTokenizer, RobertaForSequenceClassification, 
    DistilBertTokenizer, DistilBertForSequenceClassification,
    Trainer, TrainingArguments
)
from torch.utils.data import Dataset


SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -------------------------------------------------
# 1. Data Loading and Preprocessing
# -------------------------------------------------
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

def load_jsonl(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame({
        "messages": [item.get("messages", []) for item in data],
        "sender_labels": [item.get("sender_labels", []) for item in data]
    })
    # Filter out rows with empty labels
    df = df[df["sender_labels"].apply(lambda x: len(x) > 0)]
    # Combine messages
    df["messages"] = df["messages"].apply(lambda msgs: " ".join(msgs))
    # 0=Deceptive, 1=Truthful
    df["labels"] = df["sender_labels"].apply(lambda lbls: int(lbls[0]))
    return df

# Adjust your paths as needed
train_df = load_jsonl("/kaggle/input/datasetnovel1/train.jsonl")
val_df   = load_jsonl("/kaggle/input/datasetnovel1/validation.jsonl")
test_df  = load_jsonl("/kaggle/input/datasetnovel1/test.jsonl")

# Preprocess
train_df["clean_text"] = train_df["messages"].apply(preprocess_text)
val_df["clean_text"]   = val_df["messages"].apply(preprocess_text)
test_df["clean_text"]  = test_df["messages"].apply(preprocess_text)

# Upsample minority (Deceptive=0) in the training set
df_majority = train_df[train_df["labels"] == 1]
df_minority = train_df[train_df["labels"] == 0]
if len(df_minority) < len(df_majority):
    df_minority_upsampled = resample(
        df_minority, 
        replace=True,
        n_samples=len(df_majority),
        random_state=SEED  # use the fixed seed
    )
    train_df = pd.concat([df_majority, df_minority_upsampled]).sample(frac=1, random_state=SEED)

# -------------------------------------------------
# 2. Dataset Class
# -------------------------------------------------
class DeceptionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze() for key, val in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# -------------------------------------------------
# 3. Model Setup
# -------------------------------------------------
train_texts = train_df["clean_text"].tolist()
train_labels= train_df["labels"].tolist()
val_texts   = val_df["clean_text"].tolist()
val_labels  = val_df["labels"].tolist()
test_texts  = test_df["clean_text"].tolist()
test_labels = test_df["labels"].tolist()

class_weights = compute_class_weight("balanced", classes=[0,1], y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)

class WeightedRoberta(RobertaForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
            loss = loss_fn(logits, labels)
            return {"loss": loss, "logits": logits}
        return outputs

class WeightedDistilBert(DistilBertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
            loss = loss_fn(logits, labels)
            return {"loss": loss, "logits": logits}
        return outputs

roberta_tokenizer    = RobertaTokenizer.from_pretrained("roberta-base")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model_roberta = WeightedRoberta.from_pretrained("roberta-base", num_labels=2)
model_distil  = WeightedDistilBert.from_pretrained("distilbert-base-uncased", num_labels=2)

train_dataset_roberta = DeceptionDataset(train_texts, train_labels, roberta_tokenizer)
val_dataset_roberta   = DeceptionDataset(val_texts,   val_labels,   roberta_tokenizer)

train_dataset_distil  = DeceptionDataset(train_texts, train_labels, distilbert_tokenizer)
val_dataset_distil    = DeceptionDataset(val_texts,   val_labels,   distilbert_tokenizer)

# -------------------------------------------------
# 4. Trainer & Fine-Tuning
# -------------------------------------------------
torch.cuda.empty_cache()
gc.collect()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model_roberta.to(device)
model_distil.to(device)

training_args_roberta = TrainingArguments(
    output_dir="./results_roberta",
    num_train_epochs=5,               
    learning_rate=1e-5,               
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=20,
    fp16=True,
    report_to="none"
)

training_args_distil = TrainingArguments(
    output_dir="./results_distil",
    num_train_epochs=5,               
    learning_rate=1e-5,              
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=20,
    fp16=True,
    report_to="none"
)

trainer_roberta = Trainer(
    model=model_roberta,
    args=training_args_roberta,
    train_dataset=train_dataset_roberta,
    eval_dataset=val_dataset_roberta
)

trainer_distil = Trainer(
    model=model_distil,
    args=training_args_distil,
    train_dataset=train_dataset_distil,
    eval_dataset=val_dataset_distil
)

print(" Training Weighted RoBERTa...")
trainer_roberta.train()
eval_rob = trainer_roberta.evaluate()
print("Roberta eval:", eval_rob)

print("\n Training Weighted DistilBERT...")
trainer_distil.train()
eval_dis = trainer_distil.evaluate()
print("DistilBERT eval:", eval_dis)

# -------------------------------------------------
# 5. Custom Thresholding on Validation
# -------------------------------------------------
def get_probs(trainer, tokenizer, texts):
    trainer.model.eval()
    all_probs = []
    for txt in texts:
        enc = tokenizer(txt, truncation=True, padding='max_length',
                        max_length=512, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = trainer.model(**enc)
            softmax_probs = torch.nn.functional.softmax(outputs.logits, dim=1)
            all_probs.append(softmax_probs.cpu().numpy()[0])
    return np.array(all_probs)

val_probs_rob = get_probs(trainer_roberta,  roberta_tokenizer,    val_texts)
val_probs_dis = get_probs(trainer_distil,   distilbert_tokenizer, val_texts)
val_avg_probs = 0.5 * val_probs_rob + 0.5 * val_probs_dis

from sklearn.metrics import f1_score

thresholds = np.linspace(0.05, 0.95, 19)
best_thresh = 0.5
best_f1 = 0
val_labels_array = np.array(val_labels)
for t in thresholds:
    val_preds_t = (val_avg_probs[:,0] >= t).astype(int) 
    f1_0 = f1_score(val_labels_array, val_preds_t, pos_label=0)
    if f1_0 > best_f1:
        best_f1 = f1_0
        best_thresh = t

print(f"\n Found best threshold for Deceptive=0: {best_thresh:.2f} (val F1 on class 0: {best_f1:.3f})")

# -------------------------------------------------
# 6. Inference on Test Set with Threshold
# -------------------------------------------------
test_probs_rob = get_probs(trainer_roberta, roberta_tokenizer, test_df["clean_text"].tolist())
test_probs_dis = get_probs(trainer_distil,  distilbert_tokenizer, test_df["clean_text"].tolist())

test_avg_probs = 0.5 * test_probs_rob + 0.5 * test_probs_dis
test_preds = (test_avg_probs[:,0] >= best_thresh).astype(int)

# -------------------------------------------------
# 7. Final Evaluation using the Paper's Style
# -------------------------------------------------
print("\n🔍 Ensemble Model Performance on Test Set (Custom Threshold):")

report_dict = classification_report(
    test_df["labels"],
    test_preds,
    target_names=["Deceptive", "Truthful"], 
    output_dict=True
)

accuracy = report_dict["accuracy"]
macro_f1 = report_dict["macro avg"]["f1-score"]
lie_f1    = report_dict["Deceptive"]["f1-score"]
truth_f1  = report_dict["Truthful"]["f1-score"]

print(f"Accuracy:        {accuracy:.3f}")
print(f"Macro-F1:        {macro_f1:.3f}  (avg of both classes)")
print(f"Deceptive F1:    {lie_f1:.3f}    (minority class F1)")
print(f"Truthful F1:     {truth_f1:.3f}")

# print("\nFull classification report:")
# print(classification_report(
#     test_df["labels"], 
#     test_preds, 
#     target_names=["Deceptive", "Truthful"]
# ))

print("Done!")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of WeightedRoberta were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of WeightedDistilBert were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda
 Training Weighted RoBERTa...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
20,0.688800
40,0.578400
60,0.495100
80,0.374600
100,0.300000
120,0.134000
140,0.102400
160,0.102600
180,0.007100
200,0.007100


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Roberta eval: {'eval_loss': 0.7002055048942566, 'eval_runtime': 0.5895, 'eval_samples_per_second': 33.926, 'eval_steps_per_second': 5.089, 'epoch': 5.0}

 Training Weighted DistilBERT...


Step,Training Loss
20,0.695900
40,0.650800
60,0.557600
80,0.426900
100,0.303300
120,0.172600
140,0.127100
160,0.092600
180,0.054300
200,0.042000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


DistilBERT eval: {'eval_loss': 0.27284297347068787, 'eval_runtime': 0.4879, 'eval_samples_per_second': 40.996, 'eval_steps_per_second': 6.149, 'epoch': 5.0}

 Found best threshold for Deceptive=0: 0.05 (val F1 on class 0: 0.118)

🔍 Ensemble Model Performance on Test Set (Custom Threshold):
Accuracy:        0.333
Macro-F1:        0.308  (avg of both classes)
Deceptive F1:    0.176    (minority class F1)
Truthful F1:     0.440
Done!
